# Google Speech-to-Text Audio Transcripts | Google Speech-to-Text 音声書き起こし`GoogleSpeechToTextLoader` は、[Google Cloud Speech-to-Text API](https://cloud.google.com/speech-to-text)を使用して音声ファイルを文字起こしし、その文字起こししたテキストをドキュメントに読み込む機能を提供します。> The `GoogleSpeechToTextLoader` allows to transcribe audio files with the [Google Cloud Speech-to-Text API](https://cloud.google.com/speech-to-text) and loads the transcribed text into documents.これを使用するには、`google-cloud-speech` Pythonパッケージをインストールしておく必要があり、また[Speech-to-Text APIが有効になっている](https://cloud.google.com/speech-to-text/v2/docs/transcribe-client-libraries#before_you_begin)Google Cloudプロジェクトを持っている必要があります。> To use it, you should have the `google-cloud-speech` python package installed, and a Google Cloud project with the [Speech-to-Text API enabled](https://cloud.google.com/speech-to-text/v2/docs/transcribe-client-libraries#before_you_begin).* [Google CloudのSpeech APIに大規模モデルの力をもたらす](https://cloud.google.com/blog/products/ai-machine-learning/bringing-power-large-models-google-clouds-speech-api)  > [Bringing the power of large models to Google Cloud’s Speech API](https://cloud.google.com/blog/products/ai-machine-learning/bringing-power-large-models-google-clouds-speech-api)

## Installation & setup | インストールとセットアップまず、`google-cloud-speech` Pythonパッケージをインストールする必要があります。> First, you need to install the `google-cloud-speech` python package.それについての詳細は、[Speech-to-Text クライアントライブラリ](https://cloud.google.com/speech-to-text/v2/docs/libraries)のページで見つけることができます。> You can find more info about it on the [Speech-to-Text client libraries](https://cloud.google.com/speech-to-text/v2/docs/libraries) page.Google Cloudのドキュメントにある[クイックスタートガイド](https://cloud.google.com/speech-to-text/v2/docs/sync-recognize)に従って、プロジェクトを作成し、APIを有効にしてください。> Follow the [quickstart guide](https://cloud.google.com/speech-to-text/v2/docs/sync-recognize) in the Google Cloud documentation to create a project and enable the API.

In [ ]:
%pip install google-cloud-speech

## Example | 例`GoogleSpeechToTextLoader`には、`project_id`と`file_path`の引数を含める必要があります。オーディオファイルは、Google Cloud StorageのURI（`gs://...`）またはローカルファイルパスとして指定できます。> The `GoogleSpeechToTextLoader` must include the `project_id` and `file_path` arguments. Audio files can be specified as a Google Cloud Storage URI (`gs://...`) or a local file path.ローダーは同期リクエストのみをサポートしており、オーディオファイルごとに[60秒または10MBの制限](https://cloud.google.com/speech-to-text/v2/docs/sync-recognize#:~:text=60%20seconds%20and/or%2010%20MB)があります。> Only synchronous requests are supported by the loader, which has a [limit of 60 seconds or 10MB](https://cloud.google.com/speech-to-text/v2/docs/sync-recognize#:~:text=60%20seconds%20and/or%2010%20MB) per audio file.

In [2]:
from langchain.document_loaders import GoogleSpeechToTextLoader

project_id = "<PROJECT_ID>"
file_path = "gs://cloud-samples-data/speech/audio.flac"
# or a local file path: file_path = "./audio.wav"

loader = GoogleSpeechToTextLoader(project_id=project_id, file_path=file_path)

docs = loader.load()

注意：`loader.load()` を呼び出すと、転写が完了するまで処理がブロックされます。> Note: Calling `loader.load()` blocks until the transcription is finished.

`page_content`には、書き起こされたテキストが含まれています。> The transcribed text is available in the `page_content`:

In [ ]:
docs[0].page_content

```"How old is the Brooklyn Bridge?"```

`metadata`には、より多くのメタ情報を含む完全なJSONレスポンスが含まれています：> The `metadata` contains the full JSON response with more meta information:

In [ ]:
docs[0].metadata

```json{  'language_code': 'en-US',  'result_end_offset': datetime.timedelta(seconds=1)}```

## Recognition Config | 認識設定`config` 引数を指定することで、異なる音声認識モデルを使用したり、特定の機能を有効にすることができます。> You can specify the `config` argument to use different speech recognition models and enable specific features.カスタム設定を行う方法については、[Speech-to-Text recognizers documentation](https://cloud.google.com/speech-to-text/v2/docs/recognizers) および [`RecognizeRequest`](https://cloud.google.com/python/docs/reference/speech/latest/google.cloud.speech_v2.types.RecognizeRequest) APIリファレンスを参照してください。> Refer to the [Speech-to-Text recognizers documentation](https://cloud.google.com/speech-to-text/v2/docs/recognizers) and the [`RecognizeRequest`](https://cloud.google.com/python/docs/reference/speech/latest/google.cloud.speech_v2.types.RecognizeRequest) API reference for information on how to set a custom configuation.`config`を指定しない場合、以下のオプションが自動的に選択されます：> If you don't specify a `config`, the following options will be selected automatically:* モデル：[Chirp Universal Speech Model](https://cloud.google.com/speech-to-text/v2/docs/chirp-model)  > Model: [Chirp Universal Speech Model](https://cloud.google.com/speech-to-text/v2/docs/chirp-model)* <基準>  オリジナル:  言語: `en-US`  > Language: `en-US`* オーディオエンコーディング：自動検出  > Audio Encoding: Automatically Detected* 自動句読点: 有効  > Automatic Punctuation: Enabled

In [6]:
from google.cloud.speech_v2 import (
    AutoDetectDecodingConfig,
    RecognitionConfig,
    RecognitionFeatures,
)
from langchain.document_loaders import GoogleSpeechToTextLoader

project_id = "<PROJECT_ID>"
location = "global"
recognizer_id = "<RECOGNIZER_ID>"
file_path = "./audio.wav"

config = RecognitionConfig(
    auto_decoding_config=AutoDetectDecodingConfig(),
    language_codes=["en-US"],
    model="long",
    features=RecognitionFeatures(
        enable_automatic_punctuation=False,
        profanity_filter=True,
        enable_spoken_punctuation=True,
        enable_spoken_emojis=True,
    ),
)

loader = GoogleSpeechToTextLoader(
    project_id=project_id,
    location=location,
    recognizer_id=recognizer_id,
    file_path=file_path,
    config=config,
)